In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics
import matplotlib.pylab as plt
import pandas as pd
import os

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [3]:
train = pd.read_csv('FINAL_OUT_v1.csv')
train['F0823'].loc[train['F0823'] < 10.0] = -1
train['F0823'].loc[train['F0823'] >= 10.0] = 1

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
target='F0823' # F0823的值就是二元分类输出，即标签
IDcol= 'RELATED_ENODEB'
train['F0823'].value_counts()

 1.0    6857103
-1.0     585424
Name: F0823, dtype: int64

In [5]:
x_columns = [x for x in train.columns if x not in [target,IDcol]]
X = train[x_columns]
y = train[target]

In [8]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split( X, y, test_size=0.4, random_state=0)
X_train.shape, y_train.shape

((4465516, 193), (4465516,))

In [9]:
X_test.shape,y_test.shape

((2977011, 193), (2977011,))

In [1]:
#使用默认参数拟合数据
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X_train,y_train)
print rf0.oob_score_
y_predprob = rf0.predict_proba(X_train)[:,1]
print "AUC Score (Train): %f" % metrics.roc_auc_score(y_train,y_predprob)

NameError: name 'RandomForestClassifier' is not defined

In [8]:
#对n_estaimators进行网格搜索
param_test1= {'n_estimators':range(10,71,10)}
gsearch1= GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                 min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10),
                       param_grid =param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_,gsearch1.best_params_, gsearch1.best_score_
#得到最佳弱学习器迭代次数

([mean: 0.98073, std: 0.00190, params: {'n_estimators': 10},
  mean: 0.98244, std: 0.00231, params: {'n_estimators': 20},
  mean: 0.98577, std: 0.00126, params: {'n_estimators': 30},
  mean: 0.98603, std: 0.00147, params: {'n_estimators': 40},
  mean: 0.98768, std: 0.00137, params: {'n_estimators': 50},
  mean: 0.98987, std: 0.00127, params: {'n_estimators': 60},
  mean: 0.99003, std: 0.00110, params: {'n_estimators': 70}],
 {'n_estimators': 70},
 0.9900325707431648)

In [9]:
#对最大深度max_depth和min_samples_split进行网格搜索
param_test2= {'max_depth':range(3,14,2), 'min_samples_split':range(50,201,20)}
gsearch2= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60,
                                 min_samples_leaf=20,max_features='sqrt' ,oob_score=True,random_state=10),
   param_grid = param_test2,scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_,gsearch2.best_params_, gsearch2.best_score_

([mean: 0.96401, std: 0.00144, params: {'min_samples_split': 50, 'max_depth': 3},
  mean: 0.96474, std: 0.00126, params: {'min_samples_split': 70, 'max_depth': 3},
  mean: 0.96433, std: 0.00125, params: {'min_samples_split': 90, 'max_depth': 3},
  mean: 0.96422, std: 0.00116, params: {'min_samples_split': 110, 'max_depth': 3},
  mean: 0.96422, std: 0.00115, params: {'min_samples_split': 130, 'max_depth': 3},
  mean: 0.96376, std: 0.00181, params: {'min_samples_split': 150, 'max_depth': 3},
  mean: 0.96317, std: 0.00179, params: {'min_samples_split': 170, 'max_depth': 3},
  mean: 0.96311, std: 0.00184, params: {'min_samples_split': 190, 'max_depth': 3},
  mean: 0.98096, std: 0.00323, params: {'min_samples_split': 50, 'max_depth': 5},
  mean: 0.97999, std: 0.00274, params: {'min_samples_split': 70, 'max_depth': 5},
  mean: 0.97935, std: 0.00270, params: {'min_samples_split': 90, 'max_depth': 5},
  mean: 0.97775, std: 0.00152, params: {'min_samples_split': 110, 'max_depth': 5},
  mean: 0.

In [10]:
#用现有的三个参数，查看模型的袋外分数
rf1= RandomForestClassifier(n_estimators= 70, max_depth=13, min_samples_split=50,
                                 min_samples_leaf=20,max_features='sqrt' ,oob_score=True,random_state=10)
rf1.fit(X_train,y_train)
print rf1.oob_score_

0.98236546926


In [11]:
#再对内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf一起调参
param_test3= {'min_samples_split':range(50,130,20), 'min_samples_leaf':range(10,60,10)}
gsearch3= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60,max_depth=13,
                                 max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test3,scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.grid_scores_,gsearch2.best_params_, gsearch2.best_score_

([mean: 0.99465, std: 0.00041, params: {'min_samples_split': 50, 'min_samples_leaf': 10},
  mean: 0.99440, std: 0.00049, params: {'min_samples_split': 70, 'min_samples_leaf': 10},
  mean: 0.99398, std: 0.00058, params: {'min_samples_split': 90, 'min_samples_leaf': 10},
  mean: 0.99369, std: 0.00078, params: {'min_samples_split': 110, 'min_samples_leaf': 10},
  mean: 0.99446, std: 0.00028, params: {'min_samples_split': 50, 'min_samples_leaf': 20},
  mean: 0.99403, std: 0.00058, params: {'min_samples_split': 70, 'min_samples_leaf': 20},
  mean: 0.99389, std: 0.00056, params: {'min_samples_split': 90, 'min_samples_leaf': 20},
  mean: 0.99344, std: 0.00085, params: {'min_samples_split': 110, 'min_samples_leaf': 20},
  mean: 0.99420, std: 0.00059, params: {'min_samples_split': 50, 'min_samples_leaf': 30},
  mean: 0.99402, std: 0.00068, params: {'min_samples_split': 70, 'min_samples_leaf': 30},
  mean: 0.99370, std: 0.00082, params: {'min_samples_split': 90, 'min_samples_leaf': 30},
  mean: 

In [12]:
#最后我们再对最大特征数max_features做调参:
param_test4= {'max_features':range(3,11,2)}
gsearch4= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 70,max_depth=13, min_samples_split=50,
                                 min_samples_leaf=10 ,oob_score=True, random_state=10),
   param_grid = param_test4,scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.grid_scores_,gsearch4.best_params_, gsearch4.best_score_


([mean: 0.97263, std: 0.00335, params: {'max_features': 3},
  mean: 0.98666, std: 0.00123, params: {'max_features': 5},
  mean: 0.99024, std: 0.00042, params: {'max_features': 7},
  mean: 0.99270, std: 0.00032, params: {'max_features': 9}],
 {'max_features': 9},
 0.992700686431008)

In [13]:
#测试调好参数的模型袋外分数
rf2= RandomForestClassifier(n_estimators= 70, max_depth=13, min_samples_split=50,
                                 min_samples_leaf=10,max_features=9 ,oob_score=True, random_state=10)
rf2.fit(X_train,y_train)
print rf2.oob_score_

0.96481612949


In [14]:
#看测试集结果
clf = RandomForestClassifier(n_estimators= 70, max_depth=13, min_samples_split=50,
                                 min_samples_leaf=10,max_features=9 ,oob_score=True, random_state=10)
clf.fit(X_train,y_train)
print clf.score(X_test, y_test)

0.966350044725
